In [3]:
# Import necessary libraries
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import dash_table
import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from crud_module import CRUD
import plotly.graph_objects as go


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "password"
shelter = CRUD(username, password, 44883, "AAC")

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame(list(shelter.read('animals', {})))

# Limiting the number of rows displayed to 50
df = df.head(50)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add Grazioso Salvare's logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        id='header-container',
        children=[
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     style={'width': '150px', 'height': '150px', 'float': 'left'}),
            html.H2('Grazioso Salvare Dashboard', style={'padding-top': '60px', 'margin-left': '20px'}),
            html.A(html.P('Visit Grazioso Salvare', style={'font-size': '20px', 'margin-top': '70px',
                                                            'margin-right': '20px', 'float': 'right',
                                                            'text-align': 'right'}),
                   href='https://www.snhu.edu', target='_blank')
        ]
    ),
    html.Br(),
    html.Div([
        dcc.RadioItems(
            id='filter-radio',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        ),
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            page_size=10,  # Setting the page size to 10
            filter_action='native',  # Enabling filtering
            sort_action='native',  # Enabling sorting
            style_cell={'textAlign': 'left'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            }
        )
    ], className='col s12 m6'),
    html.Div([
        dcc.Graph(id='pie-chart')
    ], className='col s12 m6'),

    html.Div(
        id='map-container',
        children=[
            html.H2('Geolocation Chart'),
            html.Div(
                id='map-id',
                style={'width': '100%', 'height': '500px', 'margin-bottom': '20px'},
                children=[
                    dl.Map(id='map',
                           center=[df['location_lat'].mean(), df['location_long'].mean()],
                           zoom=10,
                           children=[
                               dl.TileLayer(),
                               dl.MarkerClusterGroup(
                                   id='marker-cluster',
                                   children=[
                                       dl.Marker(
                                           position=[row['location_lat'], row['location_long']],
                                           children=[
                                               dl.Tooltip(row['name'])
                                           ]
                                       ) for index, row in df.iterrows()
                                   ]
                               )
                           ])
                ]
            )
        ], className='col s12 m6')
    ])




############################
# Dashboard Callback / Controller
############################

@app.callback(Output('pie-chart', 'figure'),
              [Input('filter-radio', 'value')])
def update_pie_chart(value):
    if value == 'Reset':
        filtered_df = df
    else:
        filtered_df = df[df['type'] == value]
    
    # Count the number of animals by outcome type
    outcome_counts = filtered_df['outcome_type'].value_counts()
    
    # Create a Pie chart
    fig = go.Figure(data=[go.Pie(labels=outcome_counts.index,
                                 values=outcome_counts.values)])
    
    fig.update_layout(title='Outcome Types')
    
    return fig


@app.callback(Output('map', 'children'),
              [Input('filter-radio', 'value')])
def update_data(value):
    """
    Update data based on filter selection and update the geolocation chart.
    """
    if value == 'Reset':
        # If Reset selected, show all data
        filtered_df = df
    else:
        # Filter data based on selected value
        filtered_df = df[df['type'] == value]
    
    # Update geolocation chart
    children = [
        dl.Map(id='map',
               center=[filtered_df['location_lat'].mean(), filtered_df['location_long'].mean()],
               zoom=10,
               children=[
                   dl.TileLayer(),
                   dl.MarkerClusterGroup(
                       id='marker-cluster',
                       children=[
                           dl.Marker(
                               position=[row['location_lat'], row['location_long']],
                               children=[
                                   dl.Tooltip(row['name'])
                               ]
                           ) for index, row in filtered_df.iterrows()
                       ]
                   )
               ])
    ]
    
    return children

app

